In [7]:
%%capture
!pip install geopandas
!pip install sddk

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
import time

In [4]:
# import the datasets...
PHI = pd.read_json("https://zenodo.org/record/4972968/files/PHI_v1.0.json?download=1")

In [5]:
PHI_sample = PHI.sample(1000, random_state=1)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link
60228,/text/180672?location=1691&patt=&bookid=225&of...,Epigr. tou Oropou,487,"Regions\nCentral Greece (IG VII-IX)\nMegaris, ...",Epigr. tou Oropou\n487,Boiotia — Oropos (Skala) — Amphiareion — 3rd c...,"Dedication. Small base of white marble, broke...",2,1\n,[— — —]εθνη[— — —]\n[— — —] ἀνέθηκε.,...,"[[ἔθνος, ἀνατίθημι]]","[ἔθνος, ἀνατίθημι]",3rd c. BC,"{'or': None, 'not_before': -300, 'not_after': ...",-300.0,-201.0,None,"[range, cent]",None,None
159554,/text/293424?location=1703&patt=&bookid=172&bo...,SEG,32:161,"Regions\nEgypt, Nubia and CyrenaïcaEgypt and N...",SEG\n32:161,Att. — Athens — 402/1 BC,,114,stele I.1\n\n\n\n5\n\ncol. I.6\n\n\n\n10\n\n\n...,Εὐθύμαχ[ο]ς ΑΛ̣Κ̣[— — — c.14 — — — ἐγραμμάτ]-\...,...,"[[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρ...","[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρυ...",402/1 BC,"{'or': None, 'not_before': -402, 'not_after': ...",-402.0,-401.0,None,[range],None,None
209067,/text/344689?location=1665&patt=&bookid=874&of...,"IG XII,6",2:944,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,6\n2:944",Aeg. Islands — Samos Isl. — Prov. unkn. [Samos...,"Epitaph. Plaque of gray limestone, intact at ...",7,\n\n\n\n\n5\n,"vacat 0,03\n[— — — χ]αῖρε\n[— — — —]α̣ς μνή-\n...",...,"[[χαίρω, μνήμη, μήτε, ξαϲ, ρου]]","[χαίρω, μνήμη, μήτε, ξαϲ, ρου]",5th/6th c. AD,"{'or': None, 'not_before': 401, 'not_after': 6...",401.0,600.0,None,"[range, cent, morece]",None,None
149118,/text/282987?location=1617&patt=&bookid=663&of...,IK Rhod. Peraia,220,"Regions\nAsia Minor\nCaria, Rhodian Peraia",IK Rhod. Peraia\n220,"Car., Rhod.Per. — Tymnos (Bozburun) — Hellenis...",,2,1\n,Θ̣αρσύν[ο]ντι\nἥρωι.,...,"[[θαρσύνοντι, ἥρως]]","[θαρσύνοντι, ἥρως]",Hellenistic period,"{'or': None, 'source': 'PeriodO', 'notes': Non...",-330.0,-30.0,None,"[range, period]",None,http://n2t.net/ark:/99152/p03wskd55sw
52953,/text/170921?location=1687&patt=&bookid=190&of...,IK Byzantion,74,Regions\nThrace and the Lower Danube (IG X)\nT...,IK Byzantion\n74,Thrace — Byzantion (Istanbul) — 2nd c. BC — Fı...,,2,1\n,Εὐκλέων\nΜενίσκου.,...,"[[εὐκλεής, μενίσκου]]","[εὐκλεής, μενίσκου]",2nd c. BC,"{'or': None, 'not_before': -200, 'not_after': ...",-200.0,-101.0,None,"[range, cent]",None,None


In [6]:
# trismegistos api
base_url = "https://www.trismegistos.org/dataservices/texrelations/{0}?source=phi"

In [ ]:
%%time
resps = []
for n in PHI_sample["PHI_ID"][:100]:
    resp_json = requests.get(base_url.format(str(n))).json()
    resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    resps.append(resp_json)

CPU times: user 18.2 s, sys: 1.81 s, total: 20 s
Wall time: 3min 36s


In [ ]:
PHI_sample_alts = pd.DataFrame.from_records(resps)
PHI_sample_alts.head(10)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,[762219],None,None,None,None,NaN,None,NaN,[170429],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
1,[793687],None,None,None,None,NaN,None,NaN,[7984],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
2,[912191],None,None,None,None,NaN,None,NaN,[208682],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
3,[795529],None,None,None,None,NaN,None,NaN,[9829],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
4,[876295],None,None,None,None,NaN,None,NaN,[319315],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
5,[758121],None,None,None,None,NaN,None,NaN,[242840],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
6,[384368],None,None,[22200745],None,NaN,None,NaN,[344381],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
7,[851714],None,None,None,None,NaN,None,NaN,[247763],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
8,[887429],None,None,None,None,NaN,None,NaN,[296809],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None
9,[927369],None,None,None,None,NaN,None,NaN,[182681],None,...,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None


In [ ]:
def get_inscription_data(phi_id):
    try:
        resp_json = requests.get(base_url.format(str(phi_id))).json()
        resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    except:
        time.sleep(1)
        try:
            resp_json = requests.get(base_url.format(str(phi_id))).json()
            resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
        except:
            resp_json = {}
    return resp_json

In [ ]:
PHI[10:20]["PHI_ID"]


10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
Name: PHI_ID, dtype: int64

In [ ]:
%%time
### main run of the function

step=200
all_inscriptions = []
for num in range(0, len(PHI), step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<timed exec>", line 8, in <module>
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 608, in result_iterator
    yield fs.pop().result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 440, in result
    self._condition.wait(timeout)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/threading.py", line 312, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 1324, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 8, in <module>
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 636, in __exit__
    self.shutdown(wait=True)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/p

TypeError: object of type 'NoneType' has no len()

In [ ]:
all_inscriptions

[{'TM_ID': ['786251'],
  'EDB': None,
  'EDH': None,
  'EDCS': None,
  'EDR': None,
  'HE': None,
  'UOXF': None,
  'RIB': None,
  'PHI': ['1'],
  'LUPA': None,
  'ISic': None,
  'IRT': None,
  'IGCyR': None,
  'Orient': None,
  'cisp': None,
  'DeM': None,
  'PSI': None,
  'PapNav-Duke': None,
  'Lexicological_lep': None,
  'Berliner_papyri': None,
  'Bremen_papyri': None,
  'chartes': None,
  'HispEpOl': None,
  'NaBuCCo': None,
  'Ashmolean_latin_inscriptions': None,
  'PUG': None,
  'IIP': None,
  'Dresden_papyri': None,
  'GVCYR': None,
  'Vindolanda': None,
  'Ghent_papyri': None,
  'Köln_papyri': None,
  'DDBDP': None,
  'HGV': None,
  'CACHKaRN': None,
  'PapNav-Stanford': None,
  'PapNav-Fordham': None,
  'PapNav-Berenike': None,
  'PapNav-Oslo': None,
  'PapNav-Toronto': None,
  'PapNav-Lund': None,
  'PapNav-Hermitage': None,
  'PapNav-Wisconsin': None,
  'PapNav-UTS': None,
  'PapNav-UPenn': None,
  'PSR': None,
  'PTS': None,
  'PapNav-Perkins': None,
  'PapNav-Petra': Non

In [ ]:
all_inscriptions_df = pd.DataFrame(all_inscriptions)
all_inscriptions_df.head(5)

In [ ]:
all_inscriptions_df.to_json("../data/large_data/TM_PHI.json")